In [ ]:
# https://docs.opencv.org/3.4/d4/dee/tutorial_optical_flow.html#:~:text=Dense%20Optical%20Flow%20in%20OpenCV&text=Lucas%2DKanade%20method%20computes%20optical,the%20points%20in%20the%20frame.

In [ ]:
import numpy as np
import cv2

## Lucas-Kanade Optik Akış

In [ ]:
# ShiTomasi köşe tespiti için parametreler (kağıdı takip etmek için iyi özellikler)
corner_track_params = dict(maxCorners = 10,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )

### Lucas Kanade Optik Akış Parametreleri

winSize bağımsız değişkenini değiştirerek belirli noktaların hareketini veya bölgelerin toplu hareketini tespit edin. Bu, entegrasyon penceresi boyutunu belirler. Küçük pencereler gürültüye karşı daha duyarlıdır ve daha büyük hareketleri kaçırabilir. Büyük pencereler bir tıkanıklığa "hayatta kalacaktır".

Daha büyük pencere boyutuyla entegrasyon daha düzgün görünür.

burada iki kriter var - yinelemelerin maksimum sayısı (yukarıda 10) ve epsilon (yukarıda 0,03). Daha fazla yineleme, daha kapsamlı bir arama anlamına gelir ve daha küçük bir epsilon daha erken tamamlanır. Bunlar öncelikle hız ile doğruluğun değişiminde faydalıdır, ancak çoğunlukla aynı kalır.

maxLevel 0 olduğunda piramit kullanmadan (yani calcOpticalFlowLK) aynı algoritma kullanılır. Piramitler, görüntünün çeşitli çözünürlüklerinde optik akışın bulunmasını sağlar.

In [ ]:
# Lucas kanade optik akışına ilişkin parametreler
lk_params = dict( winSize  = (200,200),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10,0.03))

In [ ]:
# Videoyu yakala
cap = cv2.VideoCapture(0)

# Akışın ilk karesini yakalayın
ret, prev_frame = cap.read()

# Gri tonlamalı bir görüntü yakalayın (Buna önceki kare olarak değineceğiz)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Köşeleri yakalamak
prevPts = cv2.goodFeaturesToTrack(prev_gray, mask = None, **corner_track_params)

# Daha sonra çizim yapmak için önceki karenin eşleşen maskesini oluşturun
mask = np.zeros_like(prev_frame)


while True:

    # Geçerli kareyi yakala
    ret,frame = cap.read()

    # Gri tonlamayı yakala
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Gri Tonlamalı Çerçevede Optik Akışı Hesaplayın
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, frame_gray, prevPts, None, **lk_params)
    print(nextPts)
    # Döndürülen durum dizisini kullanma (durum çıkışı)
    # durum çıktısı durum vektörü (imzasız karakterlerin); vektörün her elemanı 1'e ayarlanır, eğer
    # karşılık gelen özelliklere ilişkin akış bulunmuştur, aksi takdirde 0'a ayarlanır.
    good_new = nextPts[status==1]
    good_prev = prevPts[status==1]

    # Çizgiler ve daireler çizmek için puan almak için ravel'ı kullanın
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):

        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()
        (x_new,y_new),(x_prev,y_prev) = (int(x_new),int(y_new)),(int(x_prev),int(y_prev))# *****************

        # İlk kareden oluşturulan maske kullanılarak çizgiler çizilecektir
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 3)

        # Köşe noktalarına kırmızı daireler çizin
        frame = cv2.circle(frame,(x_new,y_new),8,(0,0,255),-1)

    # Resmi, üzerine çizgi çizdiğimiz maskeyle birlikte görüntüleyin.
    img = cv2.add(frame,mask)
    cv2.imshow('frame',img)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Şimdi önceki kareyi ve önceki noktaları güncelleyin
    prev_gray = frame_gray.copy()
    prevPts = good_new.reshape(-1,1,2)


cv2.destroyAllWindows()
cap.release()

[[[261.76736  316.87213 ]]

 [[279.76306  316.86926 ]]

 [[316.7345   282.8754  ]]

 [[261.75934  300.87622 ]]

 [[278.75504  299.87537 ]]

 [[ 40.993313 284.85007 ]]

 [[561.80896  397.00217 ]]

 [[ 15.39743  489.6405  ]]

 [[552.79865  397.0201  ]]]


error: OpenCV(4.6.0) :-1: error: (-5:Bad argument) in function 'line'
> Overload resolution failed:
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type
>  - Can't parse 'pt1'. Sequence item with index 0 has a wrong type


# OpenCV'de Yoğun Optik Akış

calcOpticalFlowFarneback(önceki, sonraki, akış, pyr_scale, düzeyler, kazanma boyutu, yinelemeler, poly_n, poly_sigma, bayraklar) -> akış

Bu işlev, Gunnar Farneback'in algoritmasını kullanarak yoğun bir optik akışı hesaplar.

İşte fonksiyonun parametreleri ve temsil ettikleri şeyler:

* önceki ilk 8 bitlik tek kanallı giriş görüntüsü.
* öncekiyle aynı boyutta ve aynı türde sonraki ikinci giriş görüntüsü.
* öncekiyle aynı boyuta sahip ve CV_32FC2 türünde akış hesaplanmış akış görüntüsü.
* pyr_scale parametresi, her görüntü için piramitler oluşturmak amacıyla görüntü ölçeğini (\<1) belirtir
    * pyr_scale=0,5, her bir sonraki katmanın bir öncekinden iki kat daha küçük olduğu klasik bir piramit anlamına gelir.
    
* İlk görüntüyü içeren piramit katmanlarının seviye sayısı; düzeyler=1, hiçbir ekstra katmanın oluşturulmadığı ve yalnızca orijinal görüntülerin kullanıldığı anlamına gelir.
* winsize ortalama pencere boyutu
    * daha büyük değerler görüntünün algoritma sağlamlığını artırır
* gürültü ve hızlı hareket algılama için daha fazla şans verir, ancak daha fazla bulanık hareket alanı sağlar.
* yinelemeler algoritmanın her piramit seviyesinde yaptığı yinelemelerin sayısı.
* her pikseldeki polinom genişlemesini bulmak için kullanılan piksel komşuluğunun poly_n boyutu
    * daha büyük değerler, görüntünün daha pürüzsüz yüzeylerle yaklaşılacağı anlamına gelir, bu da daha sağlam bir algoritma ve daha bulanık hareket alanı sağlar, genellikle poly_n =5 veya 7.
* polinom açılımının temeli olarak kullanılan türevleri yumuşatmak için kullanılan Gaussian'ın poly_sigma standart sapması; poly_n=5 için poly_sigma=1.1'i ayarlayabilirsiniz; poly_n=7 için iyi bir değer poly_sigma=1.5 olacaktır.

In [ ]:
# Çerçeveyi yakalayın
cap = cv2.VideoCapture(0)
ret, frame1 = cap.read()

# İlk karenin gri tonlamalı görüntüsünü alın ve HSV renginde bir maske yapın
prvsImg = cv2.cvtColor(frame1,cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] = 255

while True:
    ret, frame2 = cap.read()
    nextImg = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)

    # Bu parametrelerin dökümü için yukarıdaki işaretleme metnine göz atın; bunların çoğu yalnızca önerilen varsayılanlardır.
    flow = cv2.calcOpticalFlowFarneback(prvsImg,nextImg, None, 0.5, 3, 15, 3, 5, 1.2, 0)


    # Kanalları seyahat açısına göre renklendirin
    # Videonuza çok dikkat edin; akış yönünün yolu rengi belirleyecektir!
    mag, ang = cv2.cartToPolar(flow[:,:,0], flow[:,:,1],angleInDegrees=True) # akışı kutupsal koordinatlara dönüştür
    hsv_mask[:,:,0] = ang/2
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)

    # CV'den imshow ile göstermek için BGR'ye geri dönüştürün
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    cv2.imshow('frame2',bgr)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

    # Önceki görüntüyü döngünün sonraki görüntüsü olarak ayarlayın
    prvsImg = nextImg


cap.release()
cv2.destroyAllWindows()

In [ ]:
cv2.cartToPolar

<function cartToPolar>